In [1]:
import numpy as np
import csv
from pymavlink import DFReader
from pathlib import Path

In [2]:
path = "log_bin\Drone30LOutOfControl.bin"

In [3]:
mlog = DFReader.DFReader_binary(path)
match_types = list(mlog.name_to_id.keys())
match_types.sort()
match_types.remove('FMT')
match_types.remove('UNIT')
match_types.remove('FMTU')
df_log = {}
last_timestamp = None

In [4]:
# save information to dict
'''
{'m_type' : {
            'Columns' : [col1, col2],
            'Values' : [
                [val11, val12, val13],
                [val21, val22, val23]
                ]
            }
}

'''

while True:
    m = mlog.recv_match(type=match_types)
    if m is None:
        break
    timestamp = getattr(m, '_timestamp', 0.0)
    data = m.to_dict()
    m_type = m.get_type()
    del data['mavpackettype']
    data['timestamp'] = timestamp

    if df_log.get(m_type) == None:
        df_log[m_type] = {'Columns' : list(data.keys()), 'Values' : [list(data.values())]}
    elif timestamp == last_timestamp:
        df_log[m_type]['Values'][-1] = list(data.values())
    else:
        df_log[m_type]['Values'].append(list(data.values()))

    last_timestamp = timestamp

for msg in df_log.keys():
    df_log[msg]['Values'] = np.array(df_log[msg]['Values'])

In [7]:
# save each msg to data_csv directory in csv format
path = Path(r"data_csv")
for name in match_types:
    if df_log.get(name) == None: continue
    column = df_log[name]['Columns']
    value = df_log[name]['Values']
    with open(path / str(name + ".csv"), 'w', newline='') as f:
        write = csv.writer(f)
        write.writerow(column)
        write.writerows(value)

In [8]:
with open(r"C:\Users\Lenovo\Desktop\log_analyze\models\ALL.txt") as f:
    line = f.read().splitlines()
print(line)

['battery', 'gps']
